# Word2Vec KNN

In [1]:

#import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

import time
import os
import numpy as np

import tensorflow as tf

modelfileName = "CS286-Word2Vec.model"
training_size_arr_actual = [0, 0, 0]
training_size_arr_simulated = [0, 0, 0, 0]

dirNameInput = 'Word2VecData_Input'
dirNameOutput = 'Word2VecData_Output'

#dirNameInput = 'D:\Courses\CS286 ML\Project\Code\CAN Traffic Analyzer\CAN_Traffic_Analysis\Word2Vec\Word2VecData_Input'
#dirNameOutput = 'D:\Courses\CS286 ML\Project\Code\CAN Traffic Analyzer\CAN_Traffic_Analysis\Word2Vec\Word2VecData_Output'

Word2Vec_model = None

X = None
Y = None
x_train = None
y_train = None
x_test = None
y_test = None

model = None


In [2]:
def createDataFiles():
    import time
    start = time.time()
    stride = 2
    
    fileNamesActual = ['autopark.dat', 'drive.dat', 'idle.dat']
    
    # To only work with simulated data start commenting from here
    fileInput = open("Word2VecData_Input\Word2VecDataActual.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataActual.txt", "w")
    

    for i in range(len(fileNamesActual)):
        with open(fileNamesActual[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]
        value = int(0.8 * len(content))
    
        training_size_arr_actual[i] = value
        
        value = len(content)
        print(value)

        cntCntr = 0
        while cntCntr < value:
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < value:
                parts = content[cntCntr].split(",")

                # this is the whole message
                DATA = parts[3].split(":")[1].replace(" ", "")

                currMessagePackets += str(DATA) + " "

                cntCntr += 1
                iteration += 1

            if cntCntr < value:
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(i) + "\n")
    fileInput.close()
    fileOutput.close()
    # To only work with Simulated data end commenting till here

    # To only work with actual data start commenting from here
    # For Simulated data
    fileInput = open("Word2VecData_Input\Word2VecDataSimulated.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataSimulated.txt", "w")
    fileNamesSimulated = ['New_DRIVE_Data.txt', 'New_IDLE_Data.txt', 'DRIVE.rtf', 'IDLE.rtf']
    arrSimulated = [1, 2, 1, 2]

    for i in range(len(fileNamesSimulated)):
        with open(fileNamesSimulated[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]
        
        training_size_arr_simulated[i] = (0.8 * len(content))
        
        print(len(content))

        cntCntr = 0
        while cntCntr < len(content):
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < len(content):
                # Remove the noise from the files
                DATA = content[cntCntr][15:38]

                # this is the whole message
                parts = DATA.split()

                tempo = ""
                for j in range(8):
                    if j >= len(parts) or parts[j] == "  ":
                        tempo += "00"
                    else:
                        tempo += parts[j]

                currMessagePackets += str(tempo) + " "
                cntCntr += 1
                iteration += 1

            if cntCntr < len(content):
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(arrSimulated[i]) + "\n")

    fileInput.close()
    # To only work with actual data end commenting till here
    end = time.time()

    print("Awesome !!! File processing done !!!")
    print("Total Time for file processing ---> ", end - start)


In [3]:
class IteratingClass:
    def __init__(self, dirName):
        self.dirName = dirName

    def __iter__(self):
        for fName in os.listdir(self.dirName):
            for line in open(os.path.join(self.dirName, fName)):
                yield line.split()

def createWord2VecModels():
    start = time.time()
    dataDirec = IteratingClass(dirNameInput)
    model = Word2Vec(dataDirec, size=200, window=5, min_count=1, workers=8)
    model.save(modelfileName)
    end = time.time()
    print("\n\nTraining Successful for Word2Vec Model!!!")
    print("Total Time for Word2Vec model -> ", (end - start))


In [4]:
def train():
    createDataFiles()
    createWord2VecModels()



In [5]:
train()

5592
5582
5601
29977
29993
1814
1802
Awesome !!! File processing done !!!
('Total Time for file processing ---> ', 0.7529611587524414)


Training Successful for Word2Vec Model!!!
('Total Time for Word2Vec model -> ', 1.012063980102539)


In [6]:

# Convert the data into Numpy Arrays: It will take a lot of time
start = time.time()
Word2Vec_model = Word2Vec.load(modelfileName)
x_train = None
def getNumpyArray():
    global X
    global Y
    global dirNameInput
    X = None
    Y = None
    print("Processing started ")
    for fName in os.listdir(dirNameInput):
        for line in open(os.path.join(dirNameInput, fName)):
            parts = line.split()
            currV = Word2Vec_model.wv[parts[0]]
            currV.setflags(write=1)
            for i in range(1, len(parts)):
                currV += Word2Vec_model.wv[parts[i]]
            currV /= len(parts)
            if X is None:
                X = currV
            else:
                X = np.vstack([X, currV])
                
    print("Processing started for output ")
    for fName in os.listdir(dirNameOutput):
        for line in open(os.path.join(dirNameOutput, fName)):
            currV = int(line)
            if Y is None:
                Y = currV
            else:
                Y = np.vstack([Y, currV])
            
    print(X.shape)    
    print(Y.shape)    
    end = time.time()
    print("total time -> ", end -start)





In [7]:
def createDNNModel():
    global model     
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    

    

    

In [8]:
getNumpyArray()
# Commnet this line for your part
#createDNNModel()


Processing started 
Processing started for output 
(26777, 200)
(26777, 1)
('total time -> ', 151.0931031703949)


# KNN Without Splitting Data

In [21]:
from sklearn.neighbors import KNeighborsClassifier

# KNN=1

In [24]:
knn = KNeighborsClassifier(n_neighbors=1)
print knn
knn.fit(X,np.ravel(Y,order='C'))  #train the model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [26]:
y_pred=knn.predict(X)
from sklearn import metrics
print metrics.accuracy_score(Y,y_pred)

1.0


# KNN=3

In [27]:
knn = KNeighborsClassifier(n_neighbors=3)
print knn
knn.fit(X,np.ravel(Y,order='C'))  #train the model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [28]:
y_pred=knn.predict(X)
from sklearn import metrics
print metrics.accuracy_score(Y,y_pred)

0.986891735444598


# KNN=5

In [29]:
knn = KNeighborsClassifier(n_neighbors=5)
print knn
knn.fit(X,np.ravel(Y,order='C'))  #train the model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [30]:
y_pred=knn.predict(X)
from sklearn import metrics
print metrics.accuracy_score(Y,y_pred)

0.9821862045785562


# KNN Split Data

In [22]:
# Add your code here 

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.30, random_state=1)



# KNN N = 1

In [19]:
knn = KNeighborsClassifier(n_neighbors=1)
print knn
knn.fit(x_train,np.ravel(y_train,order='C'))  #train the model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [23]:
y_pred=knn.predict(x_test)
from sklearn import metrics
print metrics.accuracy_score(y_test,y_pred)

0.9743589743589743


# KNN N = 3

In [31]:
knn = KNeighborsClassifier(n_neighbors=3)
print knn
knn.fit(x_train,np.ravel(y_train,order='C'))  #train the model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [32]:
y_pred=knn.predict(x_test)
from sklearn import metrics
print metrics.accuracy_score(y_test,y_pred)

0.9723674383868559


# KNN N = 5

In [33]:
knn = KNeighborsClassifier(n_neighbors=5)
print knn
knn.fit(x_train,np.ravel(y_train,order='C'))  #train the model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [34]:
y_pred=knn.predict(x_test)
from sklearn import metrics
print metrics.accuracy_score(y_test,y_pred)

0.9726163803833707


# KNN N = 7

In [38]:
knn = KNeighborsClassifier(n_neighbors=7)
print knn
knn.fit(x_train,np.ravel(y_train,order='C'))  #train the model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [37]:
y_pred=knn.predict(x_test)
from sklearn import metrics
print metrics.accuracy_score(y_test,y_pred)

0.9705003734129948


# KNN N = 10

In [39]:
knn = KNeighborsClassifier(n_neighbors=10)
print knn
knn.fit(x_train,np.ravel(y_train,order='C'))  #train the model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [40]:
y_pred=knn.predict(x_test)
from sklearn import metrics
print metrics.accuracy_score(y_test,y_pred)

0.97025143141648


## -------------End KNN-------------

In [10]:
def executeModel(eph):
    start = time.time()
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=eph)
    end = time.time()
    print("Total Time -> ", (end - start))
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(val_loss)
    print(val_acc)



In [11]:
start = time.time()
executeModel(50)
end = time.time()
print("Total Time -->", end - start)

AttributeError: 'NoneType' object has no attribute 'compile'

In [ ]:
score, acc = model.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
def evaluateWordToVec():
    Word2Vec_model = Word2Vec.load(modelfileName)
    print("Loaded successfully")
    #print(model.wv['FEB0FF999E500000'])
    print(len(Word2Vec_model.wv['FEB0FF999E500000']))

    #print(model.wv['000200000000002A'])
    print(len(Word2Vec_model.wv['000200000000002A']))
    
    #print(model.wv['FEB0FF999E500000'] + model.wv['000200000000002A'])
    
    v = Word2Vec_model.wv['FEB0FF999E500000'] + Word2Vec_model.wv['000200000000002A']
    print(type(v))
    #print(v/2)
evaluateWordToVec()